In [1]:
import torch
from torch.utils.data import DataLoader
from src.dataset import WikiDataset
from src.model import AgeGenderPredictor

In [2]:
# Load adience dataset for evaluation and benchmarking
dataset = WikiDataset(root_dir="./adience", tts_ratio=1)

batch_size = 8

# Put the dataset into dataloader
dataloader = DataLoader(dataset, batch_size, shuffle=True)

In [3]:
# Get the size of dataset
total = len(dataset)

print(total)

3940


In [4]:
# Specify the device to be used for inferencing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Create the model and load the trained weights
model = AgeGenderPredictor(device=device)

model.load_model("./src/checkpoints/training_4/epoch_5")

model.eval()

In [6]:
age_mae = 0
gender_mae = 0

age_accuracy = 0
gender_accuracy = 0

for data in iter(dataloader):
    # Get the test result
    age_pred, gender_pred = model.test(data)

    # Get the target expected result
    age_real, gender_real = model.age_real, model.gender_real
    
    # Calculate absolute difference
    age_mae += torch.abs(age_real - age_pred).sum().item()
    gender_mae += torch.abs(gender_real - gender_pred).sum().item()

    # Calculate number of correct predictions
    age_accuracy += (age_real == age_pred).sum().item()
    gender_accuracy += (gender_real == gender_pred).sum().item()

In [7]:
# Print out the result
print("Gender MAE: %.5f" % (gender_mae / total))
print("Gender accuracy: %.2f" % (gender_accuracy / total * 100), "%")
print("Age MAE: %.5f" % (age_mae / total))
print("Age accuracy: %.2f" % (age_accuracy / total * 100), "%")

Gender MAE: 0.49086
Gender accuracy: 50.91 %
Age MAE: 1.71066
Age accuracy: 3.86 %
